In [ ]:
!pip install wikipedia sentence_transformers python-dotenv

In [2]:
!pip install langchain-community

  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_core-0.3.78-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached sqlalchemy-2.0.43-cp312-cp312-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.13.0-cp312-cp312-win_amd64.whl.metadata (8.4 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.11.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached langsmith-0.4.33-py3-none-any.whl.metadata (14 kB)
  Using cached httpx_sse-0.4.2-py3-none-any.whl.metadata (9.5 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.8.0-cp312-cp312-win_amd64.whl.metadata (21 kB)
  Using cache


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import wikipedia
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

load_dotenv()  # loads .env

# Path to store Chroma DB
CHROMA_PATH = Path("chroma_db")
CHROMA_PATH.mkdir(exist_ok=True)

f:\GEN-AI NOTES\LangChain\Projects\4_NewRAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def load_local_docs(folder: str = "data") -> list:
    """Load all .txt and .pdf files from a folder."""
    docs = []
    for file in Path(folder).glob("*"):
        if file.suffix == ".txt":
            loader = TextLoader(str(file))
        elif file.suffix == ".pdf":
            loader = PyPDFLoader(str(file))
        else:
            continue
        docs.extend(loader.load())
    return docs

In [6]:
from langchain_core.documents import Document

In [7]:
def load_wikipedia_articles(titles: list[str]) -> list:
    """Fetch Wikipedia pages and return as Documents."""
    docs = []
    for title in titles:
        try:
            page = wikipedia.page(title)
            content = page.content
            docs.append(
                Document(page_content=content, metadata={"source": f"Wikipedia:{title}"})
            )
        except Exception as e:
            print(f"Failed to load {title}: {e}")
    return docs

In [7]:

%pip install langchain_huggingface
from langchain_huggingface import HuggingFaceEmbeddings

  Using cached langchain_huggingface-0.3.1-py3-none-any.whl.metadata (996 bytes)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached langchain_huggingface-0.3.1-py3-none-any.whl (27 kB)
Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl (2.7 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
def create_chroma_store(docs: list, persist_path: Path = CHROMA_PATH):
    """Create a Chroma vector store from documents."""
    # Use Groq embeddings (free tier)
    embeddings =HuggingFaceEmbeddings(
        model="sentence-transformers/all-mpnet-base-v2"
    )
    # Persist the store
    db = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=str(persist_path),
    )
    db.persist()
    print(f"Chroma DB stored at {persist_path}")

In [ ]:
# if __name__ == "__main__":
#     # 1️⃣ Load local docs
#     local_docs = load_local_docs()

#     # 2️⃣ Load Wikipedia articles (optional)
#     wiki_titles = ["Python (programming language)", "LangChain", "Groq"]
#     wiki_docs = load_wikipedia_articles(wiki_titles)

#     # 3️⃣ Combine
#     all_docs = local_docs + wiki_docs

#     # 4️⃣ Create Chroma store
#     create_chroma_store(all_docs)

In [12]:
local_docs = load_local_docs()

In [19]:
create_chroma_store(local_docs)

Chroma DB stored at chroma_db


C:\Users\jhota\AppData\Local\Temp\ipykernel_19420\3725697247.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [8]:
load_wikipedia_articles(["Python (programming language)", "LangChain", "Groq"])

Failed to load LangChain: Page id "lang chain" does not match any pages. Try another id!


[Document(metadata={'source': 'Wikipedia:Python (programming language)'}, page_content='Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.\nPython is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.\nGuido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language. Python 3.0, released in 2008, was a major revision and not completely backward-compatible with earlier versions. Recent versions, such as Python 3.13, 3.12 and older (and 3.14), have added capabilities and keywords for typing (and more; e.g. increasing speed); helping with (optional) static typing. Currently only versions in the 3.x series are supported.\nPython consistently ranks as one of the most popular programming languages, and it has gained wide